# import requests

[Pocket Developer Program: Pocket Authentication API: Documentation](https://getpocket.com/developer/docs/authentication)

In [1]:
import json
import yaml
import requests

headers = {
    "Content-Type":"application/json;charset=UTF-8"
    , "X-Accept": "application/json"
}

with open(".private.yaml") as text:
    data = yaml.load(text)
    consumer_key = data["consumer_key"]
    redirect_uri = data["redirect_uri"]
    code = data["code"]

# Request code

codeはUser名的な位置づけなので、一度取得すればよい。
一度ブラウザから"url"へアクセスしアプリケーションの使用を許可する必要がある

In [5]:
import os

In [14]:
data = {
    "consumer_key":consumer_key
    , "redirect_uri":redirect_uri
}
r = requests.post("https://getpocket.com/v3/oauth/request",data=json.dumps(data),headers=headers)

code = json.loads(r.text)["code"]

url = "https://getpocket.com/auth/authorize?request_token=%s&redirect_uri=%s" % (code, data["redirect_uri"])
url
os.system("open '%s'" % url)

0

In [15]:
code

'58f1f695-3138-53ef-d9fb-51bb42'

# Authorize

取得したUser(code)から、Authorizeを実行し、短時間有効なaccess_tokenを取得

In [16]:
data = {
    "consumer_key":consumer_key
    ,"code": code
}
r = requests.post("https://getpocket.com/v3/oauth/authorize",data=json.dumps(data),headers=headers)
access_token = json.loads(r.text)["access_token"]

# Retrieve

[Pocket Developer Program: Pocket API: Retrieve](https://getpocket.com/developer/docs/v3/retrieve)

In [17]:
data = {
    "consumer_key": consumer_key
    , "access_token": access_token
    , "count":"5000"
    , "sort": "newest"
    , "detailType":"complete"
}

r = requests.post("https://getpocket.com/v3/get",data=json.dumps(data),headers=headers)

In [18]:
result = json.loads(r.text)

In [19]:
import random
import datetime

In [20]:
quarters = [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12]]

def mecab_text(quarter):
    keys = [key for key in result["list"].keys() if datetime.date.fromtimestamp(int(result["list"][key]["time_added"])).year == 2015
           and datetime.date.fromtimestamp(int(result["list"][key]["time_added"])).month in quarter]

    title_list = []
    for key in keys:
        try:
            title_list.append("%s"%result["list"][key]["resolved_title"])
        except:
            title_list.append("%s"%result["list"][key]["given_title"])
    return " ".join(title_list)
texts = []
for quarter in quarters:
    texts.append(mecab_text(quarter))

In [21]:
import MeCab
import collections
def word_ranking(text):
    mecab = MeCab.Tagger ("-Ochasen")
    mecab_result = mecab.parse(text)
    temp = [line.split("\t") for line in mecab_result.split("\n")]
    del temp[-2:]
    word = [line[0] for line in temp if "名詞" in line[3] and len(line[0]) >=3 ]
    data = collections.Counter(word)
    return [line for line in data.most_common() if line[1] >= 3]

In [22]:
ranking = []
for text in texts:
    ranking.append(word_ranking(text))

In [23]:
i = 1
for monthly_ranking in ranking:
    print("%i月:%s" % (i,monthly_ranking[0:5]))
    i = i + 1

1月:[('Ruby', 20), ('Rails', 13), ('Web', 11), ('ツール', 10), ('サイト', 10)]
2月:[('データ', 6), ('Python', 5), ('サービス', 4), ('The', 4), ('and', 4)]
3月:[('データ', 12), ('デザイン', 7), ('Deck', 6), ('プログラミング', 6), ('Speaker', 6)]
4月:[('データ', 7), ('2015', 5), ('サイト', 5), ('ハッカー', 4), ('まとめ', 4)]
5月:[('データ', 5), ('ビジネス', 4), ('2015', 4), ('システム', 4), ('マンション', 4)]
6月:[('AWS', 3), ('エンジニア', 3)]
7月:[('Amazon', 12), ('Data', 9), ('エンジニア', 8), ('AWS', 7), ('Elastic', 6)]
8月:[('Python', 12), ('2015', 9), ('データ', 8), ('Web', 7), ('プログラミング', 5)]
9月:[('データ', 12), ('Python', 10), ('AWS', 5), ('インテル', 3), ('Data', 3)]
10月:[('AWS', 12), ('Python', 10), ('データ', 10), ('エンジニア', 7), ('2015', 7)]
11月:[('AWS', 17), ('2015', 10), ('Python', 6), ('IoT', 6), ('Lambda', 6)]
12月:[('AWS', 6), ('Python', 4), ('デザイン', 3), ('初心者', 3), ('Google', 3)]


In [27]:
import pprint
pprint.pprint(os.environ["PATH"])

'/usr/local/heroku/bin:/Library/Frameworks/Python.framework/Versions/3.4/bin:/Users/kkuma/.rvm/gems/ruby-2.0.0-p451/bin:/Users/kkuma/.rvm/gems/ruby-2.0.0-p451@global/bin:/Users/kkuma/.rvm/rubies/ruby-2.0.0-p451/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/Users/kkuma/.rvm/bin:/Users/kkuma/.rvm/bin'
